# Contraction Expansion

## Params to change

In [1]:
num_file = 0
sliding_window_len = 120
rolling_mean_smoothing_window = 5
savgol_window_len = 70
savgol_polyorder = 3

In [2]:
import pandas as pd
from pathlib import Path

bilateral_dir = Path().cwd()
txt_files = sorted(bilateral_dir.glob("*.txt"))
df = pd.read_csv(txt_files[num_file], skiprows=4, sep="\t")
list(zip(range(len(df.columns)),df.columns))
cols_to_keep = [col for col in df.columns if (col.endswith("x") or col.endswith("y") or col.endswith("z")) and not "speed" in col and not "dist" in col][2:]
df = df[cols_to_keep].drop_duplicates()
df.head()

,Elbowloutx,Elbowlouty,Elbowloutz,Elbowroutx,Elbowrouty,Elbowroutz,Hipoutx,Hipouty,Hipoutz,Kneesoutx,...,Neckoutz,Rightwristoutx,Rightwristouty,Rightwristoutz,Shoulderloutx,Shoulderlouty,Shoulderloutz,Shoulderroutx,Shoulderrouty,Shoulderroutz
0,0.145375,-0.479709,3.129696,0.600708,-0.472405,3.140847,0.385976,-0.874493,3.095856,0.382623,...,3.217612,0.518592,-0.561945,2.974752,0.212695,-0.276518,3.201952,0.549337,-0.274441,3.211237
1,0.145259,-0.480681,3.129265,0.600999,-0.472245,3.141013,0.385986,-0.874463,3.095749,0.382821,...,3.217571,0.519795,-0.562454,2.974838,0.213397,-0.275654,3.200079,0.549146,-0.274404,3.211368
2,0.145508,-0.480570,3.129879,0.600911,-0.470547,3.143299,0.385964,-0.874530,3.095803,0.382767,...,3.217818,0.521381,-0.561770,2.976878,0.212886,-0.275705,3.200647,0.549135,-0.274649,3.211206
4,0.145632,-0.480455,3.129338,0.602356,-0.464212,3.147738,0.385962,-0.874502,3.095752,0.382755,...,3.217828,0.524768,-0.558887,2.982373,0.212950,-0.275737,3.200651,0.549129,-0.274551,3.211205
6,0.145437,-0.480655,3.129632,0.601745,-0.467905,3.142447,0.385969,-0.874465,3.095763,0.382727,...,3.217854,0.521078,-0.560266,2.977221,0.212855,-0.275830,3.200998,0.549244,-0.274449,3.211327


In [3]:
import numpy as np
print("Before cleaning:")
if df.isna().any(axis=None) or (df == 0.0).any(axis=None):
    print("\033[91mThe dataframe contains NaN or zero values.\033[0m")
    
df.replace(0, np.nan, inplace=True)
df.interpolate(method='linear', inplace=True)
df.ffill(inplace=True)
df.bfill(inplace=True)  # Backfill to handle any remaining NaNs

print("After cleaning:")
if df.isna().any(axis=None) or (df == 0.0).any(axis=None):
    print("\033[91mWARNING: The dataframe contains NaN or zero values.\033[0m")
else:
    print("\033[92mThe dataframe does not contain NaN or zero values.\033[0m")

Before cleaning:
The dataframe contains NaN or zero values.
After cleaning:
The dataframe does not contain NaN or zero values.


In [5]:
markersPosTableX = df.iloc[:,0::3]
markersPosTableZ = df.iloc[:,1::3]
markersPosTableY = df.iloc[:,2::3]

markersPosTableX = markersPosTableX.rename(columns=dict(zip(list(markersPosTableX),[name.replace("out","") for name in list(markersPosTableX)])))
markersPosTableY = markersPosTableY.rename(columns=dict(zip(list(markersPosTableY),[name.replace("out","") for name in list(markersPosTableY)])))
markersPosTableZ = markersPosTableZ.rename(columns=dict(zip(list(markersPosTableZ),[name.replace("out","") for name in list(markersPosTableZ)])))
markersPosTableX = markersPosTableX.reindex(sorted(markersPosTableX.columns), axis=1)
markersPosTableY = markersPosTableY.reindex(sorted(markersPosTableY.columns), axis=1)
markersPosTableZ = markersPosTableZ.reindex(sorted(markersPosTableZ.columns), axis=1)
markers_names = [col[:-1] for col in markersPosTableX.columns]
print("Markers names:", markers_names)
colors = ["blue"]*len(markers_names)

markersPosX_no_smooth = markersPosTableX.values
markersPosY_no_smooth = markersPosTableY.values
markersPosZ_no_smooth = markersPosTableZ.values

from scipy.signal import savgol_filter

# Apply smoothing to the marker positions
markersPosX = savgol_filter(markersPosTableX.rolling(window=rolling_mean_smoothing_window, min_periods=1, center=True).mean().values, window_length=savgol_window_len, polyorder=savgol_polyorder, axis=0)
markersPosY = savgol_filter(markersPosTableY.rolling(window=rolling_mean_smoothing_window, min_periods=1, center=True).mean().values, window_length=savgol_window_len, polyorder=savgol_polyorder, axis=0)
markersPosZ = savgol_filter(markersPosTableZ.rolling(window=rolling_mean_smoothing_window, min_periods=1, center=True).mean().values, window_length=savgol_window_len, polyorder=savgol_polyorder, axis=0)

# Calculate the center of mass for each frame
center_of_mass = np.array([markersPosX.mean(axis=1), markersPosY.mean(axis=1), markersPosZ.mean(axis=1)]).T

Markers names: ['Elbowl', 'Elbowr', 'Hip', 'Knees', 'Leftwrist', 'Neck', 'Rightwrist', 'Shoulderl', 'Shoulderr']


In [6]:
import numpy as np

def get_adjacency_matrix(numMarkers, bones):
    edges = np.array(bones)
    adjacencyMatrix = np.zeros((numMarkers,numMarkers),dtype=bool)
    adjacencyMatrix[edges[:,0],edges[:,1]] = True
    return adjacencyMatrix

In [ ]:
from pyeyesweb.low_level.contraction_expansion import ContractionExpansion
from pyeyesweb.data_models.sliding_window import SlidingWindow

# Calculate the bilateral symmetry of the two hands using the BilateralSymmetry
sw = SlidingWindow(max_length=sliding_window_len, n_columns=3, m_joints=markersPosX.shape[1])
contraction_expansion = ContractionExpansion()

values = {(left, right) : {
    'bilateral_symmetry_index': np.zeros(markersPosX.shape[0]),
    'cca_correlation': np.zeros(markersPosX.shape[0])
} for left, right in}

for frame in range(markersPosX.shape[0]):
    sw.append([markersPosX[frame], markersPosY[frame], markersPosZ[frame]])
    res = contraction_expansion(sw)
    for (left, right), metrics in res.items():
        values[(left, right)]['bilateral_symmetry_index'][frame] = metrics['bilateral_symmetry_index']
        values[(left, right)]['cca_correlation'][frame] = metrics['cca_correlation']

/home/gaggio/miniconda3/envs/pyeyesweb/lib/python3.11/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)
/home/gaggio/miniconda3/envs/pyeyesweb/lib/python3.11/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)
/home/gaggio/miniconda3/envs/pyeyesweb/lib/python3.11/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)
/home/gaggio/miniconda3/envs/pyeyesweb/lib/python3.11/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)
/home/gaggio/miniconda3/envs/pye

In [21]:
values_to_plot = [values[(7,8)][feature_to_plot], values[(0,1)][feature_to_plot], values[(4,6)][feature_to_plot]]
colors_to_plot = [colors[7], colors[0], colors[4]]

In [ ]:
%matplotlib qt

from matplotlib import pyplot as plt
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib.widgets import Button
import matplotlib.animation as animation
#bones = infer_bones(df)
import os

import json
import time

#bones = infer_bones(df)
#print(bones)

if 'anim' in globals() and anim.event_source != None:
    anim.pause()

def get_adjacency_matrix(numMarkers, bones):
    edges = np.array(bones)
    adjacencyMatrix = np.zeros((numMarkers,numMarkers),dtype=bool)
    if bones == []:
        return adjacencyMatrix
    adjacencyMatrix[edges[:,0],edges[:,1]] = True
    return adjacencyMatrix

# Convert bones into adjacency matrix
bones = [[4,0],[0,7],[7,8],[8,1],[1,6],[5,2],[2,3]]
adjacencyMatrix = get_adjacency_matrix(markersPosX.shape[1], bones)

fig = plt.figure("3D Movement", figsize=(19.20, 10.80))
plt.clf()

# Add a 3D subplot with the specified position and size
ax = fig.add_axes([0, -0.16, 1, .85], projection='3d')
ax.view_init(elev=0, azim=-90)
# Create three stacked subplots for the synchronization plots
left, bottom, width, total_height = 0.05, 0.55, 0.9, 0.42
gap = 0.04
height = (total_height - 2 * gap) / 3

ax_acc1 = fig.add_axes([left, bottom + 2 * (height + gap), width, height])
ax_acc2 = fig.add_axes([left, bottom + (height + gap), width, height], sharex=ax_acc1)
ax_acc3 = fig.add_axes([left, bottom, width, height], sharex=ax_acc1)


ax_acc1.plot(values_to_plot[0], color=colors_to_plot[0])
ax_acc2.plot(values_to_plot[1], color=colors_to_plot[1])
ax_acc3.plot(values_to_plot[2], color=colors_to_plot[2])

ax_acc1.legend([f"{markers_names[joint_pairs[0][0]]}-{markers_names[joint_pairs[0][1]]}"], loc='best')
ax_acc2.legend([f"{markers_names[joint_pairs[1][0]]}-{markers_names[joint_pairs[1][1]]}"], loc='best')
ax_acc3.legend([f"{markers_names[joint_pairs[2][0]]}-{markers_names[joint_pairs[2][1]]}"], loc='best')
ax_acc1.set_yticks([0, 0.7, 1])
ax_acc2.set_yticks([0, 0.7, 1])
ax_acc3.set_yticks([0, 0.7, 1])
ax_acc1.set_xticks([])  # Hide x-axis ticks for the first two subplots
ax_acc2.set_xticks([])  # Hide x-axis ticks for the first two subplots
ax_acc3.set_xticks(list(range(0, len(values_to_plot[2]), 300)))  # Show x-axis ticks for the last subplot
ax_acc1_set_ylim = ax_acc1.set_ylim(-0.05, 1.05)
ax_acc2_set_ylim = ax_acc2.set_ylim(-0.05, 1.05)
ax_acc3_set_ylim = ax_acc3.set_ylim(-0.05, 1.05)
#ax_acc.plot(speed, color='green')
#ax_acc.set_yticks(np.linspace(0,max(acceleration),10).tolist()+ [max(acceleration)+0.05, max(acceleration)+0.55])
#ax_acc.set_yticklabels(np.round(np.linspace(0,max(acceleration),10),3).tolist()+ ["v = 0", f"v = {max(speed):.2f}"])
ax_acc1.set_title(f'Feature: {feature_to_plot} | Signal: {signal} | Sliding window len = {sliding_window_len}', fontsize=10)
ax_acc3.set_xlabel('F r a m e')
#ax_acc.grid(axis='y', color='gray')
ax_acc2.set_ylabel('Synchronization')

#ax_acc.plot(peaks, acceleration[peaks], 'ro', markersize=5, label='Peaks')
ax_acc1.grid(axis='y', color='gray')
ax_acc2.grid(axis='y', color='gray')
ax_acc3.grid(axis='y', color='gray')

cursor1 = ax_acc1.plot(0, values_to_plot[0][0], 'bo', markersize=4, label='Current Frame')
cursor2 = ax_acc2.plot(0, values_to_plot[1][0], 'bo', markersize=4, label='Current Frame')
cursor3 = ax_acc3.plot(0, values_to_plot[2][0], 'bo', markersize=4, label='Current Frame')
cursor = [cursor1, cursor2, cursor3]

# Calculate movement boundaries for the view
minMax = np.zeros((2,3))
minMax[0,0] = np.nanmin(markersPosX[3:])
minMax[0,1] = np.nanmin(markersPosY[3:])
minMax[0,2] = np.nanmin(markersPosZ[3:])
minMax[1,0] = np.nanmax(markersPosX[3:])
minMax[1,1] = np.nanmax(markersPosY[3:])
minMax[1,2] = np.nanmax(markersPosZ[3:])
#minMax = np.array([[-2292.197, -2213.251, 15.017],[ 617.451, 760.748, 1815.185]])


# Set appropriate axis limits
ax.set_xlim([minMax[0,0],minMax[1,0]])
ax.set_ylim([minMax[0,1],minMax[1,1]])
ax.set_zlim([minMax[0,2],minMax[1,2]])

# Set the window title
#fig.canvas.manager.window.title("3D Movement")


# Set background white and grid lines white to mimic transparency
plt.rcParams['grid.color'] = 'white'
ax.tick_params(axis='both', colors='white')

# Remove box faces except the ground
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = True
#ax.w_zaxis.pane.set_facecolor("w")
ax.xaxis.pane.set_edgecolor("w")
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor("w")

# Hide axis lines and ticks
ax.xaxis.line.set_visible(False)
ax.yaxis.line.set_visible(False)
ax.zaxis.line.set_visible(False) 


def update_plot(val):
    #for annotation in ax.texts:
    #    annotation.remove()

    # filter the data based on the current time index
    filteredX = markersPosX[val]
    filteredY = markersPosY[val]
    filteredZ = markersPosZ[val]
    
    # Update scatter plot data instead of creating a new scatter plot
    scatter_plot = ax.collections[0]
    scatter_plot.set_offsets(np.column_stack((filteredX, filteredY)))
    scatter_plot._offsets3d = (filteredX, filteredY, filteredZ)

    # Update bones based on the adjacency matrix
    k = 0
    for i in range(len(adjacencyMatrix)):
        for j in range(0, len(adjacencyMatrix)):
            if adjacencyMatrix[i, j]:

                # Find the existing line corresponding to this edge and update its coordinates
                line = allLines[k]
                line.set_xdata([filteredX[i], filteredX[j]])
                line.set_ydata([filteredY[i], filteredY[j]])
                line.set_3d_properties([filteredZ[i], filteredZ[j]])
                k+=1
    
    # Add labels for each marker
    #for idx, (x, y, z) in enumerate(zip(filteredX, filteredY, filteredZ)):
    #    ax.text(x, y, z, str(idx), color='black', fontsize=14)

    for id, cursor in enumerate([cursor1, cursor2, cursor3]):
        cursor[0].set_xdata([val])
        cursor[0].set_ydata([values_to_plot[id][val]])
    
    fig.canvas.draw_idle()


filteredX = markersPosX[0]
filteredY = markersPosY[0]
filteredZ = markersPosZ[0]

allLines = []

# Plot the first scatters
#print(markersPosX[0].shape, markersPosY[0].shape, markersPosZ[0].shape)
ax.scatter(markersPosX[0],markersPosY[0],markersPosZ[0], s=60, c=colors)
for i in range(len(adjacencyMatrix)):
    for j in range(len(adjacencyMatrix)):
        if adjacencyMatrix[i,j]:
            ax.plot([filteredX[i], filteredX[j]],
                    [filteredY[i], filteredY[j]],
                    [filteredZ[i], filteredZ[j]],
                    color='k', linestyle='-', linewidth=0.7)
            allLines.append(ax.lines[-1])


# Create a slider widget
slider_ax = plt.axes([0.08, 0, 0.8, 0.03])
maxValue = len(markersPosX)-1
slider = Slider(slider_ax, 'Frame:', 0, maxValue, valinit=0, valstep=1)

# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Function to update the plot for animation
def animate(frame):
    slider.set_val(frame)

step = 1
anim = animation.FuncAnimation(fig, slider.set_val, frames=range(0, maxValue, step), interval=1, repeat=True, cache_frame_data=False)

last_ = (time.perf_counter(), 0)
remaining_time = [1]

def _print_progress(current_frame, total_frames):
    global last_
    now = time.perf_counter()
    dt = now - last_[0]
    last_frame = last_[1]
    last_ = (now, current_frame)
    if last_frame % 5 == 2:
        remaining_time.append(dt * (total_frames - current_frame))
        if len(remaining_time) > 100:
            remaining_time.pop(0)
        avg_remaining_time = sum(remaining_time) / len(remaining_time)
        print(f"Saving frame {current_frame}/{total_frames} | remaining_time={avg_remaining_time:.4f}s", end="\r")

anim.save(bilateral_dir / f"result_{txt_files[num_file].stem}_{feature_to_plot}.mp4", writer='ffmpeg', fps=60, progress_callback=_print_progress)


# Create a button to pause and play the animation
button_ax = plt.axes([0.92, 0.005, 0.06, 0.03])
button = Button(button_ax, 'Pause/Play')
running = True

# Function to pause and play the animation
def pause_play(event):
    global running 
    if running:
        anim.pause()
    else: 
        anim.resume()
    running ^= True

button.on_clicked(pause_play)


# Show the plot
plt.show()
